In [1]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tabulate import tabulate
import numpy as np
import IPython.display as d

In [2]:
# --- US company list

response = requests.get('https://financialmodelingprep.com/api/v3/stock/list?apikey=f33b3631d5140a4f1c87e7f2eafd8fdd')
response.encoding = 'utf-8'
raw_list=response.json()
comp_list = [i['symbol'] for i in raw_list]
exchange = set([i['exchange'] for i in raw_list])
exchange_us = ['NASDAQ',
 'NASDAQ Global Market',
 'NYSE',
 'NYSE American',
 'Nasdaq',
 'Nasdaq Capital Market',
 'Nasdaq Global Market',
 'Nasdaq Global Select',
 'New York Stock Exchange',
 'New York Stock Exchange Arca']
us_list = [i['symbol'] for i in raw_list if i['exchange'] in exchange_us]

In [3]:
def file(n):
    with open(f'data/{n}.txt', encoding = 'utf-8') as f:
        data = json.load(f)
        f.close()
    return data

In [4]:
asset = file('asset')
income = file('income')
cashflow = file('cashflow')
growth = file('growth')
ratio = file('ratio')
profile = file('profile')

In [ ]:
def checker(n):
    temp = []
    temp2 = []
    for i in n:
        if i != None:
            for j in i:
                if j != None:
                    temp.append(j['period'])
                    temp2.append(j['date'][0:4])
    return Counter(temp), Counter(temp2)

checker(growth)


In [ ]:
# --- Preprocessing profile.txt

temp = []
for i in profile:
    for j in i:
        if j != 0 :
            temp.append(j)  
# with open('data/profile.txt', 'w') as f:
#     json.dump(temp, f)
df = pd.DataFrame(temp)
df.head()


In [33]:
# --- Setup a company name list
df_new = df[['symbol', 'companyName', 'currency', 'exchange', 'exchangeShortName', 'country', 'state']]
df_new.to_excel('data\company_list.xlsx')

In [41]:
# --- Add a time series tag for quarter

def preprocessing(n: list):

    temp = []
    for i in n:
        if i != None:
            for j in i:
                if len(j['date']) == 10 and j['period'] != None and j['period'].startswith('Q') and len(j['period']) == 2:
                    year = int(j['date'][0:4])
                    quarter = int(j['period'][-1])
                    j['year'] = year
                    j['quarter'] = (year - 2001)*4 + quarter
                    temp.append(j)
                else:
                    i.remove(j)
    return temp

asset2 = preprocessing(asset)
income2 = preprocessing(income)
cashflow2 = preprocessing(cashflow)
growth2 = preprocessing(growth)
ratio2 = preprocessing(ratio)

    

In [40]:
asset_df.iloc[0]

date                                                                              2021-03-31
symbol                                                                                 CMCSA
reportedCurrency                                                                         USD
fillingDate                                                                       2021-04-29
acceptedDate                                                             2021-04-29 15:22:23
period                                                                                    Q1
cashAndCashEquivalents                                                           14950000000
shortTermInvestments                                                                       0
cashAndShortTermInvestments                                                      14950000000
netReceivables                                                                    1.0986e+10
inventory                                                             

In [42]:
# ---- After finishing: convert to dataframe

asset_df = pd.DataFrame(asset2)
asset_df.to_csv('data/asset_df.csv')
income_df = pd.DataFrame(income2)
income_df.to_csv('data/income_df.csv')
cashflow_df = pd.DataFrame(cashflow2)
cashflow_df.to_csv('data/cashflow_df.csv')    
growth_df = pd.DataFrame(growth2)
growth_df.to_csv('data/growth_df.csv')
ratio_df = pd.DataFrame(ratio2)
ratio_df.to_csv('data/ratio_df.csv')

In [ ]:
asset_2 = asset_df.loc[asset_df['date'] == 2020]
asset_2.head()

In [42]:
combined_df = pd.merge(asset_df, income_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, cashflow_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, growth_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, ratio_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))

In [35]:
irrelevant = ['link','finalLink', 'goodwillAndIntangibleAssets', 'epsdiluted', 'weightedAverageShsOutDil','operatingCashFlow','capitalExpenditure']

In [43]:
combined_df.drop(columns=irrelevant, inplace=True)
combined_df.drop([i for i in combined_df.columns if 'drop' in i], axis=1, inplace=True)
combined_df['date'] = pd.to_datetime(combined_df['date'])
combined_df.head()


,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,2021-03-31,CMCSA,USD,2021-04-29,2021-04-29 15:22:23,Q1,14950000000,0,14950000000,1.098600e+10,...,9.131373,18.655678,47.049056,32.049931,32.049931,-17.455952,9.131373,0.004347,47.118783,2.683435
1,2020-12-31,CMCSA,USD,2021-02-04,2021-02-03 18:08:14,Q4,11740000000,0,11740000000,1.146600e+10,...,8.662711,17.753431,157.601046,47.605393,47.605393,0.263889,8.662711,0.004391,55.651505,2.616434
2,2020-09-30,CMCSA,USD,2020-10-29,2020-10-29 15:03:25,Q1,13707000000,0,13707000000,1.031000e+10,...,8.292809,26.217436,92.499786,40.499620,40.499620,-0.819628,8.292809,0.004997,48.797840,2.417833
3,2020-06-30,CMCSA,USD,2020-07-30 00:00:00,2020-07-30 18:31:48,Q2,13935000000,0,13935000000,1.022700e+10,...,7.572459,15.025172,30.100714,20.777607,20.777607,0.380639,7.572459,0.005853,34.187316,2.117924
4,2020-03-31,CMCSA,USD,2020-04-30 00:00:00,2020-04-30 15:24:15,Q1,8516000000,0,8516000000,1.080000e+10,...,5.965368,18.483054,47.739087,27.254887,27.254887,-0.577117,5.965368,0.006155,31.270757,1.917452


In [44]:
# --- Validation of data

test = combined_df[['date','symbol','year','period','quarter']]
test.head()

,date,symbol,year,period,quarter
0,2021-03-31,CMCSA,2021,Q1,81
1,2020-12-31,CMCSA,2020,Q4,80
2,2020-09-30,CMCSA,2020,Q1,77
3,2020-06-30,CMCSA,2020,Q2,78
4,2020-03-31,CMCSA,2020,Q1,77


In [53]:
test['Quarter'] = test['date'].apply(lambda x: int(int(x.split('-')[1])/3))
test['Validate'] = (test['period'].apply(lambda x: int(x[-1])) == test['Quarter'])
test.groupby('Validate')['Validate'].count()

<ipython-input-53-fcee33656cba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Quarter'] = test['date'].apply(lambda x: int(int(x.split('-')[1])/3))
<ipython-input-53-fcee33656cba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Validate'] = (test['period'].apply(lambda x: int(x[-1])) == test['Quarter'])


Validate
False     76111
True     220628
Name: Validate, dtype: int64

In [46]:
# --- Remaking correct quarter number

for i in [asset_df, income_df,cashflow_df,ratio_df,growth_df]:
    i['year'] = i['date'].apply(lambda x: int(int(x.split('-')[0])))
    i['quarter'] = i['date'].apply(lambda x: int(int(x.split('-')[1])/3)) + (i['year']-2001)*4
asset_df.head()

,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,...,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,totalInvestments,totalDebt,netDebt,link,finalLink,quarter,year
0,2021-03-31,CMCSA,USD,2021-04-29,2021-04-29 15:22:23,1,14950000000,0,14950000000,1.098600e+10,...,0.0,9.257500e+10,2.769860e+11,0,103713000000,88763000000,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,81,2021
1,2020-12-31,CMCSA,USD,2021-02-04,2021-02-03 18:08:14,4,11740000000,0,11740000000,1.146600e+10,...,0.0,9.173800e+10,2.738690e+11,0,103760000000,92020000000,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,80,2020
2,2020-09-30,CMCSA,USD,2020-10-29,2020-10-29 15:03:25,3,13707000000,0,13707000000,1.031000e+10,...,0.0,8.757100e+10,2.689400e+11,0,104424000000,90717000000,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,79,2020
3,2020-06-30,CMCSA,USD,2020-07-30 00:00:00,2020-07-30 18:31:48,2,13935000000,0,13935000000,1.022700e+10,...,0.0,8.479100e+10,2.659780e+11,0,104810000000,90875000000,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/Archives/edgar/data/116669...,78,2020
4,2020-03-31,CMCSA,USD,2020-04-30 00:00:00,2020-04-30 15:24:15,1,8516000000,0,8516000000,1.080000e+10,...,0.0,8.278300e+10,2.624210e+11,0,103577000000,95061000000,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/Archives/edgar/data/116669...,77,2020


In [49]:
income_df.head()

,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,revenue,costOfRevenue,grossProfit,grossProfitRatio,...,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,link,finalLink,quarter,year
0,2021-03-31,CMCSA,USD,2021-04-29,2021-04-29,1,2.720500e+10,8.919000e+09,1.828600e+10,0.672156,...,3.329000e+09,0.122367,0.730000,0.710000,4.591000e+09,4.665000e+09,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,81,2021
1,2020-12-31,CMCSA,USD,2021-02-04,2021-02-04,4,2.770800e+10,9.438000e+09,1.827000e+10,0.659376,...,3.380000e+09,0.121986,0.737886,0.730000,4.580656e+09,4.645000e+09,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,80,2020
2,2020-09-30,CMCSA,USD,2020-10-29,2020-10-29,3,2.553200e+10,8.565000e+09,1.696700e+10,0.664539,...,2.019000e+09,0.079077,0.441119,0.436258,4.577000e+09,4.628000e+09,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,79,2020
3,2020-06-30,CMCSA,USD,2020-07-30 00:00:00,2020-07-30 18:31:48,2,2.371500e+10,6.817000e+09,1.689800e+10,0.712545,...,2.988000e+09,0.125996,0.648580,0.648580,4.607000e+09,4.607000e+09,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/Archives/edgar/data/116669...,78,2020
4,2020-03-31,CMCSA,USD,2020-04-30 00:00:00,2020-04-30 15:24:15,1,2.660900e+10,8.301000e+09,1.830800e+10,0.688038,...,2.147000e+09,0.080687,0.465020,0.465020,4.617000e+09,4.617000e+09,https://www.sec.gov/Archives/edgar/data/116669...,https://www.sec.gov/Archives/edgar/data/116669...,77,2020


In [47]:
combined_df = pd.merge(asset_df, income_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, cashflow_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, growth_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, ratio_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))

In [50]:
asset_df.to_csv('data/asset_df.csv')
income_df.to_csv('data/income_df.csv')
cashflow_df.to_csv('data/cashflow_df.csv')    
growth_df.to_csv('data/growth_df.csv')
ratio_df.to_csv('data/ratio_df.csv')
combined_df.to_csv('data/combined.csv')

In [5]:
combined_df = pd.read_csv('data/combined.csv')
